In [1]:
#UTILS
# !pip install visdom
import random
import time
import datetime
import sys

from torch.autograd import Variable
import torch
# from visdom import Visdom
import numpy as np

def print_network(net):
    num_params = 0
    for param in net.parameters():
        num_params += param.numel()
    print(net)
    print('Total number of parameters: %d' % num_params)

def tensor2image(tensor):
    image = 127.5*(tensor[0].cpu().float().numpy() + 1.0)
    if image.shape[0] == 1:
        image = np.tile(image, (3,1,1))
    return image.astype(np.uint8)

# class Logger():
#     def __init__(self, n_epochs, batches_epoch):
#         self.viz = Visdom()
#         self.n_epochs = n_epochs
#         self.batches_epoch = batches_epoch
#         self.epoch = 1
#         self.batch = 1
#         self.prev_time = time.time()
#         self.mean_period = 0
#         self.losses = {}
#         self.loss_windows = {}
#         self.image_windows = {}


    def log(self, losses=None, images=None):
        self.mean_period += (time.time() - self.prev_time)
        self.prev_time = time.time()

        sys.stdout.write('\rEpoch %03d/%03d [%04d/%04d] -- ' % (self.epoch, self.n_epochs, self.batch, self.batches_epoch))

        for i, loss_name in enumerate(losses.keys()):
            if loss_name not in self.losses:
                self.losses[loss_name] = losses[loss_name].data[0]
            else:
                self.losses[loss_name] += losses[loss_name].data[0]

            if (i+1) == len(losses.keys()):
                sys.stdout.write('%s: %.4f -- ' % (loss_name, self.losses[loss_name]/self.batch))
            else:
                sys.stdout.write('%s: %.4f | ' % (loss_name, self.losses[loss_name]/self.batch))

        batches_done = self.batches_epoch*(self.epoch - 1) + self.batch
        batches_left = self.batches_epoch*(self.n_epochs - self.epoch) + self.batches_epoch - self.batch 
        sys.stdout.write('ETA: %s' % (datetime.timedelta(seconds=batches_left*self.mean_period/batches_done)))

        # Draw images
        for image_name, tensor in images.items():
            if image_name not in self.image_windows:
                self.image_windows[image_name] = self.viz.image(tensor2image(tensor.data), opts={'title':image_name})
            else:
                self.viz.image(tensor2image(tensor.data), win=self.image_windows[image_name], opts={'title':image_name})

        # End of epoch
        if (self.batch % self.batches_epoch) == 0:
            # Plot losses
            for loss_name, loss in self.losses.items():
                if loss_name not in self.loss_windows:
                    self.loss_windows[loss_name] = self.viz.line(X=np.array([self.epoch]), Y=np.array([loss/self.batch]), 
                                                                    opts={'xlabel': 'epochs', 'ylabel': loss_name, 'title': loss_name})
                else:
                    self.viz.line(X=np.array([self.epoch]), Y=np.array([loss/self.batch]), win=self.loss_windows[loss_name], update='append')
                # Reset losses for next epoch
                self.losses[loss_name] = 0.0

            self.epoch += 1
            self.batch = 1
            sys.stdout.write('\n')
        else:
            self.batch += 1

        

class ReplayBuffer():
    def __init__(self, max_size=50):
        assert (max_size > 0), 'Empty buffer or trying to create a black hole. Be careful.'
        self.max_size = max_size
        self.data = []

    def push_and_pop(self, data):
        to_return = []
        for element in data.data:
            element = torch.unsqueeze(element, 0)
            if len(self.data) < self.max_size:
                self.data.append(element)
                to_return.append(element)
            else:
                if random.uniform(0,1) > 0.5:
                    i = random.randint(0, self.max_size-1)
                    to_return.append(self.data[i].clone())
                    self.data[i] = element
                else:
                    to_return.append(element)
        return Variable(torch.cat(to_return))

class LambdaLR():
    def __init__(self, n_epochs, offset, decay_start_epoch):
        assert ((n_epochs - decay_start_epoch) > 0), "Decay must start before the training session ends!"
        self.n_epochs = n_epochs
        self.offset = offset
        self.decay_start_epoch = decay_start_epoch

    def step(self, epoch):
        return 1.0 - max(0, epoch + self.offset - self.decay_start_epoch)/(self.n_epochs - self.decay_start_epoch)

def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        torch.nn.init.normal(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm2d') != -1:
        torch.nn.init.normal(m.weight.data, 1.0, 0.02)
#        torch.nn.init.constant(m.bias.data, 0.0)

In [2]:
#DATASET

import glob
import random
import os

from torch.utils.data import Dataset
from PIL import Image
import torchvision.transforms as transforms

#A: Cloud
#B: Clear
#We want to go from A to B



class CloudClearDataset(Dataset):
    def __init__(self, root, transforms_=None, unaligned=False, mode='train'):
        self.transform = transforms.Compose(transforms_)
        self.unaligned = unaligned

        self.files_Cloud = sorted(glob.glob(os.path.join(root,'%s/Cloud' % mode) + '/*.png'))
        self.files_Clear = sorted(glob.glob(os.path.join(root,'%s/Clear' % mode) + '/*.png'))

    def __getitem__(self, index):
        item_Cloud = self.transform(Image.open(self.files_Cloud[index % len(self.files_Cloud)]))

        if self.unaligned:
            item_Clear = self.transform(Image.open(self.files_Clear[random.randint(0, len(self.files_Clear) - 1)]))
        else:
            item_Clear = self.transform(Image.open(self.files_Clear[index % len(self.files_Clear)]))

        return {'Cloud': item_Cloud, 'Clear': item_Clear}

    def __len__(self):
        return max(len(self.files_Clear), len(self.files_Clear))

In [3]:
#MODELS.py


import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F

class Generator(nn.Module):
    def __init__(self, input_nc=3, output_nc=4, ngf=64, norm_layer=nn.BatchNorm2d, use_dropout=False, n_blocks=6):
        assert(n_blocks >= 0)
        super(Generator, self).__init__()
        self.input_nc = input_nc
        self.output_nc = output_nc
        self.ngf = ngf

        model = [nn.ReflectionPad2d(3),
                 nn.Conv2d(input_nc, ngf, kernel_size=7, padding=0),
                 norm_layer(ngf),
                 nn.ReLU(True)]

        n_downsampling = 2
        for i in range(n_downsampling):
            mult = 2**i
            model += [nn.Conv2d(ngf * mult, ngf * mult * 2, kernel_size=3,
                                stride=2, padding=1),
                      norm_layer(ngf * mult * 2),
                      nn.ReLU(True)]

        mult = 2**n_downsampling
        for i in range(n_blocks):
            model += [ResnetBlock(ngf * mult, norm_layer=norm_layer, use_dropout=use_dropout)]

        for i in range(n_downsampling):
            mult = 2**(n_downsampling - i)
            model += [nn.ReflectionPad2d(1),
                      nn.Conv2d(ngf * mult, int(ngf * mult / 2),
                                kernel_size=3, stride=1),
                      norm_layer(int(ngf * mult / 2)),
                      nn.ReLU(True),
                      nn.Conv2d(int(ngf * mult / 2), int(ngf * mult / 2)*4,
                                kernel_size=1, stride=1),
                      nn.PixelShuffle(2),
                      norm_layer(int(ngf * mult / 2)),
                      nn.ReLU(True),
                     ]
        model += [nn.ReflectionPad2d(3)]
        model += [nn.Conv2d(ngf, output_nc, kernel_size=7, padding=0)]

        self.model = nn.Sequential(*model)

    def forward(self, input):
        output = self.model(input)
        attention_mask = F.sigmoid(output[:, :1])
        content_mask = output[:, 1:]
        attention_mask = attention_mask.repeat(1, 3, 1, 1)
        result = content_mask * attention_mask + input * (1 - attention_mask)

        return result, attention_mask, content_mask

class ResnetBlock(nn.Module):
    def __init__(self, dim, norm_layer, use_dropout):
        super(ResnetBlock, self).__init__()
        self.conv_block = self.build_conv_block(dim, norm_layer, use_dropout)

    def build_conv_block(self, dim, norm_layer, use_dropout):
        conv_block = [nn.ReflectionPad2d(1),
                       nn.Conv2d(dim, dim, kernel_size=3),
                       norm_layer(dim),
                       nn.ReLU(True)]
        if use_dropout:
            conv_block += [nn.Dropout(0.5)]

        conv_block += [nn.ReflectionPad2d(1),
                       nn.Conv2d(dim, dim, kernel_size=3),
                       norm_layer(dim)]

        return nn.Sequential(*conv_block)

    def forward(self, x):
        out = x + self.conv_block(x)
        return out


class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.conv_tower = nn.Sequential(
            nn.Conv2d(3,   64,  4, 2),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(),
            nn.Conv2d(64,  128,  4, 2),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(),
            nn.Conv2d(128,  256,  4, 2),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(),
            nn.Conv2d(256,  512, 4, 2),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(),
            nn.Conv2d(512, 512, 4),
            nn.LeakyReLU(),
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Conv2d(512, 1, 1),
        )

    def forward(self, img):
        output = self.conv_tower(img)
        return output


In [ ]:
#!/usr/bin/python3

import argparse
import itertools
import sys
import os

import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn as nn
from torch.autograd import Variable
from PIL import Image
import torch
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision.utils import save_image
import torchvision

# from models import Generator
# from models import Discriminator
# from utils import ReplayBuffer
# from utils import LambdaLR
# from utils import Logger
# from utils import weights_init_normal
# from utils import print_network
# from dataset import CloudClearDataset


#GLOBAL VARIABLES
epoch = 0
n_epochs = 60
batchSize = 1
dataroot = '/content/drive/MyDrive/CS-MultiMedia/AI/Project/rice_dataset/'
save_name = '/content/drive/MyDrive/CS-MultiMedia/AI/Project/Inference/'
lr = 0.0001
decay_epoch = 10
size = 512
input_nc = 3
output_nc = 3
# cuda = 
n_cpu = 8
lambda_cycle = 10
lambda_identity = 0
lambda_pixel = 1
lambda_reg = 1e-6
gan_curriculum = 10
starting_rate = 0.01
default_rate = 0.5



# parser = argparse.ArgumentParser()
# parser.add_argument('--epoch', type=int, default=0, help='starting epoch')
# parser.add_argument('--n_epochs', type=int, default=200, help='number of epochs of training')
# parser.add_argument('--batchSize', type=int, default=1, help='size of the batches')
# parser.add_argument('--dataroot', type=str, default='/content/drive/MyDrive/CS-MultiMedia/AI/Project/rice_dataset/', help='root directory of the dataset')
# parser.add_argument('--save_name', type=str, default='attentionGAN_RICE1_CloudClear')
# parser.add_argument('--lr', type=float, default=0.0001, help='initial learning rate')
# parser.add_argument('--decay_epoch', type=int, default=100, help='epoch to start linearly decaying the learning rate to 0')
# parser.add_argument('--size', type=int, default=256, help='size of the data crop (squared assumed)')
# parser.add_argument('--input_nc', type=int, default=3, help='number of channels of input data')
# parser.add_argument('--output_nc', type=int, default=3, help='number of channels of output data')
# parser.add_argument('--cuda', action='store_true', help='use GPU computation')
# parser.add_argument('--n_cpu', type=int, default=8, help='number of cpu threads to use during batch generation')
# parser.add_argument('--lambda_cycle', type=int, default=10)
# parser.add_argument('--lambda_identity', type=int, default=0)
# parser.add_argument('--lambda_pixel', type=int, default=1)
# parser.add_argument('--lambda_reg', type=float, default=1e-6)

# parser.add_argument('--gan_curriculum', type=int, default=10, help='Strong GAN loss for certain period at the beginning')
# parser.add_argument('--starting_rate', type=float, default=0.01, help='Set the lambda weight between GAN loss and Recon loss during curriculum period at the beginning. We used the 0.01 weight.')
# parser.add_argument('--default_rate', type=float, default=0.5, help='Set the lambda weight between GAN loss and Recon loss after curriculum period. We used the 0.5 weight.')

# opt = parser.parse_args()
# print(opt)

# if torch.cuda.is_available():
#     print("WARNING: You have a CUDA device, so you should probably run with --cuda")

###### Definition of variables ######
# Networks
netG_Cloud2Clear = Generator()
netG_Clear2Cloud = Generator()
netD_Cloud = Discriminator()
netD_Clear = Discriminator()

print('---------- Networks initialized -------------')
print_network(netG_Cloud2Clear)
print_network(netG_Clear2Cloud)
print_network(netD_Cloud)
print_network(netD_Clear)
print('-----------------------------------------------')

# if opt.cuda:
netG_Cloud2Clear.cuda()
netG_Clear2Cloud.cuda()
netD_Cloud.cuda()
netD_Clear.cuda()

netG_Cloud2Clear.apply(weights_init_normal)
netG_Clear2Cloud.apply(weights_init_normal)
netD_Cloud.apply(weights_init_normal)
netD_Clear.apply(weights_init_normal)

# Lossess
criterion_GAN = torch.nn.MSELoss()
criterion_cycle = torch.nn.L1Loss()
criterion_identity = torch.nn.L1Loss()

# Optimizers & LR schedulers
optimizer_G = torch.optim.Adam(itertools.chain(netG_Cloud2Clear.parameters(), netG_Clear2Cloud.parameters()),
                                lr=lr, betas=(0.5, 0.999))
optimizer_D = torch.optim.Adam(itertools.chain(netD_Cloud.parameters(), netD_Clear.parameters()),
                                lr=lr, betas=(0.5, 0.999))

lr_scheduler_G = torch.optim.lr_scheduler.LambdaLR(optimizer_G, lr_lambda=LambdaLR(n_epochs, epoch, decay_epoch).step)
lr_scheduler_D = torch.optim.lr_scheduler.LambdaLR(optimizer_D, lr_lambda=LambdaLR(n_epochs, epoch, decay_epoch).step)

# Inputs & targets memory allocation
Tensor = torch.cuda.FloatTensor #if opt.cuda else torch.Tensor
input_Cloud = Tensor(batchSize, input_nc, size, size)
input_Clear = Tensor(batchSize, output_nc, size, size)
target_real = Variable(Tensor(batchSize).fill_(1.0), requires_grad=False)
target_fake = Variable(Tensor(batchSize).fill_(0.0), requires_grad=False)

fake_Cloud_buffer = ReplayBuffer()
fake_Clear_buffer = ReplayBuffer()

# Dataset loader
transforms_ = [ transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize([0.5,0.5,0.5], [0.5,0.5,0.5]) ]

dataloader = DataLoader(CloudClearDataset(dataroot, transforms_=transforms_, unaligned=True), 
                        batch_size=batchSize, shuffle=True, num_workers=n_cpu)

# Loss plot
# logger = Logger(n_epochs, len(dataloader)) # IT HAS AN ERROR WE HAVE TO PLOT THE LOSS LATER

###### Training ######
for epoch in range(epoch, n_epochs):
    for i, batch in enumerate(dataloader):
        # Set model input
        real_Cloud = Variable(input_Cloud.copy_(batch['Cloud']))
        real_Clear = Variable(input_Clear.copy_(batch['Clear']))

        ###### Generators A2B and B2A ######
        optimizer_G.zero_grad()

        # Identity loss
        # G_A2B(B) should equal B if real B is fed
        same_Clear, _, _ = netG_Cloud2Clear(real_Clear)
        loss_identity_Clear = criterion_identity(same_Clear, real_Clear)*lambda_identity
        # G_B2A(A) should equal A if real A is fed
        same_Cloud, _, _ = netG_Clear2Cloud(real_Cloud)
        loss_identity_Cloud = criterion_identity(same_Cloud, real_Cloud)*lambda_identity

        # GAN loss
        fake_Clear, mask_Clear, temp_Clear = netG_Cloud2Clear(real_Cloud)
        recovered_A, _, _ = netG_Clear2Cloud(fake_Clear)
        pred_fake_Clear = netD_Clear(fake_Clear)

        loss_cycle_CloudClearCloud = criterion_cycle(recovered_A, real_Cloud)
        loss_GAN_Cloud2Clear = criterion_GAN(pred_fake_Clear, target_real)
        loss_pix_Cloud = criterion_identity(fake_Clear, real_Cloud)

        fake_Cloud, mask_Cloud, temp_Cloud = netG_Clear2Cloud(real_Clear)
        recovered_B, _, _  = netG_Cloud2Clear(fake_Cloud)
        pred_fake_Cloud = netD_Cloud(fake_Cloud)

        loss_cycle_ClearCloudClear = criterion_cycle(recovered_B, real_Clear)
        loss_GAN_Clear2Cloud = criterion_GAN(pred_fake_Cloud, target_real)
        loss_pix_Clear = criterion_identity(fake_Cloud, real_Clear)

        loss_reg_Cloud = lambda_reg * (
                torch.sum(torch.abs(mask_Cloud[:, :, :, :-1] - mask_Cloud[:, :, :, 1:])) +
                torch.sum(torch.abs(mask_Cloud[:, :, :-1, :] - mask_Cloud[:, :, 1:, :])))

        loss_reg_Clear = lambda_reg * (
                torch.sum(torch.abs(mask_Clear[:, :, :, :-1] - mask_Clear[:, :, :, 1:])) +
                torch.sum(torch.abs(mask_Clear[:, :, :-1, :] - mask_Clear[:, :, 1:, :])))

        # Total loss
        if epoch < gan_curriculum:
            rate = starting_rate
            print('using curriculum gan')
        else:
            rate = default_rate
            print('using normal gan')

        loss_G = ((loss_GAN_Cloud2Clear + loss_GAN_Clear2Cloud)*0.5 + (loss_reg_Cloud + loss_reg_Clear))* (1.-rate) + ((loss_cycle_CloudClearCloud + loss_cycle_ClearCloudClear)*lambda_cycle+(loss_pix_Clear+loss_pix_Cloud)*lambda_pixel)* rate
    
        loss_G.backward()
        optimizer_G.step()
        ###################################

        optimizer_D.zero_grad()

        # Real loss
        pred_real_Cloud = netD_Cloud.forward(real_Cloud)
        loss_D_real_Cloud = criterion_GAN(pred_real_Cloud, target_real)

        # Fake loss
        fake_Cloud = fake_Cloud_buffer.push_and_pop(fake_Cloud)
        pred_fake_Cloud = netD_Cloud.forward(fake_Cloud.detach())
        loss_D_fake_Cloud = criterion_GAN(pred_fake_Cloud, target_fake)

        # Real loss
        pred_real_Clear = netD_Clear.forward(real_Clear)
        loss_D_real_Clear = criterion_GAN(pred_real_Clear, target_real)
        
        # Fake loss
        fake_Clear = fake_Clear_buffer.push_and_pop(fake_Clear)
        pred_fake_Clear = netD_Clear.forward(fake_Clear.detach())
        loss_D_fake_Clear = criterion_GAN(pred_fake_Clear, target_fake)

        # Total loss
        loss_D = (loss_D_real_Clear + loss_D_fake_Clear + loss_D_real_Cloud + loss_D_fake_Cloud)*0.25
        
        loss_D.backward()
        optimizer_D.step()

        print('Epoch [%d/%d], Batch [%d/%d], loss_D: %.4f, loss_G: %.4f' % (epoch+1, n_epochs,i+1, len(dataloader), loss_D.data, loss_G.data)) 
        print('loss_GAN_Cloud2Clear: %.4f, loss_GAN_Clear2Cloud: %.4f, loss_cycle_CloudClearCloud: %.4f, loss_cycle_ClearCloudClear: %.4f, loss_identity_Cloud: %.4f, loss_identity_Clear: %.4f, loss_pix_Cloud: %.4f, loss_pix_Clear: %.4f' % (loss_GAN_Cloud2Clear.data, 
            loss_GAN_Clear2Cloud.data, loss_cycle_CloudClearCloud.data, loss_cycle_ClearCloudClear.data, loss_identity_Cloud.data, loss_identity_Clear.data, loss_pix_Cloud.data, loss_pix_Clear.data)) 

        save_path='%s/%s' % (save_name, 'training')
        if not os.path.exists(save_path):
            os.makedirs(save_path)

        save_image(torch.cat([
            real_Cloud.data.cpu()[0] * 0.5 + 0.5,
            mask_Clear.data.cpu()[0],
            fake_Clear.data.cpu()[0]*0.5+0.5, temp_Clear.data.cpu()[0]*0.5+0.5], 2),
            '%s/%04d_%04d_progress_B.png' % (save_path,epoch+1,i+1))

        save_image(torch.cat([
            real_Clear.data.cpu()[0] * 0.5 + 0.5,
            mask_Cloud.data.cpu()[0],
            fake_Cloud.data.cpu()[0]*0.5+0.5, temp_Clear.data.cpu()[0]*0.5+0.5], 2),
            '%s/%04d_%04d_progress_A.png' % (save_path,epoch+1,i+1))

    # Update learning rates
    lr_scheduler_G.step()
    lr_scheduler_D.step()

    torch.save(netG_Cloud2Clear.state_dict(), '%s/%s' % (save_name, 'netG_Cloud2Clear.pth'))
    torch.save(netG_Clear2Cloud.state_dict(), '%s/%s' % (save_name, 'netG_Clear2Cloud.pth'))
    torch.save(netD_Cloud.state_dict(), '%s/%s' % (save_name, 'netD_Cloud.pth'))
    torch.save(netD_Clear.state_dict(), '%s/%s' % (save_name, 'netD_Clear.pth'))

---------- Networks initialized -------------
Generator(
  (model): Sequential(
    (0): ReflectionPad2d((3, 3, 3, 3))
    (1): Conv2d(3, 64, kernel_size=(7, 7), stride=(1, 1))
    (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): ReLU(inplace=True)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (8): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): ResnetBlock(
      (conv_block): Sequential(
        (0): ReflectionPad2d((1, 1, 1, 1))
        (1): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1))
        (2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (3): ReLU(inplace=True)
        (4): ReflectionPad2d

<ipython-input-1-31317655205d>:123: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  torch.nn.init.normal(m.weight.data, 0.0, 0.02)
<ipython-input-1-31317655205d>:125: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  torch.nn.init.normal(m.weight.data, 1.0, 0.02)


using curriculum gan
Epoch [1/60], Batch [1/450], loss_D: 0.9869, loss_G: 2.3385
loss_GAN_Cloud2Clear: 1.1135, loss_GAN_Clear2Cloud: 2.4037, loss_cycle_CloudClearCloud: 0.5184, loss_cycle_ClearCloudClear: 0.4538, loss_identity_Cloud: 0.0000, loss_identity_Clear: 0.0000, loss_pix_Cloud: 0.4150, loss_pix_Clear: 0.3081
using curriculum gan
Epoch [1/60], Batch [2/450], loss_D: 0.5325, loss_G: 0.5415
loss_GAN_Cloud2Clear: 0.0519, loss_GAN_Clear2Cloud: 0.0489, loss_cycle_CloudClearCloud: 0.4326, loss_cycle_ClearCloudClear: 0.6214, loss_identity_Cloud: 0.0000, loss_identity_Clear: 0.0000, loss_pix_Cloud: 0.3852, loss_pix_Clear: 0.4037
using curriculum gan
Epoch [1/60], Batch [3/450], loss_D: 0.4788, loss_G: 0.5044
loss_GAN_Cloud2Clear: 0.0755, loss_GAN_Clear2Cloud: 0.0196, loss_cycle_CloudClearCloud: 0.3635, loss_cycle_ClearCloudClear: 0.5620, loss_identity_Cloud: 0.0000, loss_identity_Clear: 0.0000, loss_pix_Cloud: 0.3639, loss_pix_Clear: 0.3739
using curriculum gan
Epoch [1/60], Batch [4/45